In [1]:
# Pruebas de seguimiento solar y demanda neta

# Importamos librerías
import os
import rasterio
import numpy                           as np
import pandas                          as pd
import geopandas                       as gpd
import xarray                          as xr
from   matplotlib        import pyplot as plt
from   scipy             import optimize
from   matplotlib.ticker import StrMethodFormatter

# Funciones trigonométricas.
def sin(x) : return np.sin(np.radians(x))
def cos(x) : return np.cos(np.radians(x))
def tan(x) : return np.tan(np.radians(x))
def asin(x): return np.arcsin(x) * 180/np.pi
def acos(x): return np.arccos(x) * 180/np.pi
def atan(x): return np.arctan(x) * 180/np.pi

# Huso horario.
# La información está en UTC.
TZ = -6

In [2]:
# Datos de radiación

# Rutas de archivos
path_f = "../data/"
name_f = "NSRDB_4km_2022_00.nc"

# Cargamos archivos de radiación
ds = xr.open_dataset(path_f + name_f)

# Convertimos a fecha.
ds["hour"] = ( ds["time"].dt.hour + ds["time"].dt.minute / 60 ).copy()
ds["dayofyear"] = ds["time"].dt.dayofyear.copy()

In [3]:
def NUM_CENACE(x):
    if   x == "BCA": y = 0
    elif x == "BCS": y = 1
    elif x == "CEN": y = 2
    elif x == "NES": y = 3
    elif x == "NOR": y = 4
    elif x == "NTE": y = 5
    elif x == "OCC": y = 6
    elif x == "ORI": y = 7
    elif x == "PEN": y = 8
    return y

regiones = pd.read_csv( "../data/regiones_CENACE.csv",
    index_col = "CLAVE ENTIDAD" )
estados = gpd.read_file("../data/Mapas/dest23gw")
estados["CVE_ENT"] = estados["CVE_ENT"].astype(int)
estados = estados.set_index("CVE_ENT")
estados["REG_CENACE"] = regiones["REGION CENACE"]
estados["CVE_CENACE"] = regiones["CLAVE CENACE"]
estados["NUM_CENACE"] = regiones["CLAVE CENACE"].apply(NUM_CENACE)

points = ds.isel(time = 0)["Temperature"].to_dataframe(
    ).reset_index().drop( columns = ["time", "Temperature"] )
points["geometry"] = gpd.points_from_xy(points["lon"], points["lat"])
points = gpd.GeoDataFrame(points)
points["CVE_ENT"] = np.nan
for cve in estados.index:
    points["MASK_ENT"] = points.within(estados.loc[cve, "geometry"])
    points["CVE_ENT"] = points["CVE_ENT"].where( ~points["MASK_ENT"], cve )

points = pd.merge(points, estados["NUM_CENACE"].reset_index(), on = "CVE_ENT")

CENACE = points.set_index( ["lon", "lat"]
    )[ ["CVE_ENT", "NUM_CENACE"] ].to_xarray()

ds = xr.merge([ds, CENACE])
ds

<xarray.Dataset> Size: 80MB
Dimensions:      (time: 8760, lon: 16, lat: 13)
Coordinates:
  * time         (time) datetime64[ns] 70kB 2022-01-01 ... 2022-12-31T23:00:00
  * lon          (lon) float64 128B -100.5 -100.4 -100.4 ... -99.94 -99.9 -99.86
  * lat          (lat) float64 104B 18.09 18.13 18.17 ... 18.49 18.53 18.57
Data variables:
    Temperature  (time, lat, lon) float64 15MB ...
    DNI          (time, lat, lon) float64 15MB ...
    GHI          (time, lat, lon) float64 15MB ...
    Pressure     (time, lat, lon) float64 15MB ...
    Wind Speed   (time, lat, lon) float64 15MB ...
    UVHI         (time, lat, lon) float32 7MB ...
    hour         (time) float64 70kB 0.0 1.0 2.0 3.0 4.0 ... 20.0 21.0 22.0 23.0
    dayofyear    (time) int64 70kB 1 1 1 1 1 1 1 ... 365 365 365 365 365 365 365
    CVE_ENT      (lon, lat) float64 2kB 12.0 12.0 12.0 12.0 ... 12.0 12.0 12.0
    NUM_CENACE   (lon, lat) int64 2kB 7 7 7 7 7 7 7 7 7 7 ... 7 7 7 7 7 7 7 7 7
Attributes:
    CDI:                       Climate Data Interface version 2.4.0 (https://...
    Conventions:               CF-1.6
    history:                   Fri Sep 06 19:15:50 2024: cdo distgrid,5,5 NSR...
    cdo_openmp_thread_number:  2
    CDO:                       Climate Data Operators version 2.4.0 (https://...

In [4]:
# Cálculo de posición solar

lat = ds["lat"]
lon = ds["lon"]

# Eccentric anomaly of the earth in its orbit around the sun.
ds["Day_Angle"] = 6.283185 * ( ds["dayofyear"] - 1 ) / 365
# Declinación.
ds["Declination"] = ( ( 0.006918 - 0.399912 * np.cos(ds["Day_Angle"])
    + 0.070257*np.sin(ds["Day_Angle"])
    - 0.006758*np.cos(2*ds["Day_Angle"])
    + 0.000907*np.sin(2*ds["Day_Angle"])
    - 0.002697*np.cos(3*ds["Day_Angle"])
    + 0.00148*np.sin(3*ds["Day_Angle"]) ) * 180/np.pi )
# Ecuación del tiempo.
ds["Time_Equation"] = ( ( 0.000075 + 0.001868*np.cos(ds["Day_Angle"])
    - 0.032077*np.sin(ds["Day_Angle"])
    - 0.014615*np.cos(2*ds["Day_Angle"])
    -0.040849*np.sin(2*ds["Day_Angle"])) * 229.18 )
# Longitud del punto subsolar.
ds["lon_subs"] = -15 * ( ds["hour"] - TZ
    + ds["Time_Equation"]/60 )
# Ángulo horario.
ds["Hour_Angle"] = ( 15 * ( ds["hour"] - 12
    - ds["Time_Equation"]/60 + ((lon-TZ*15)*4)/60 ) )
ds = ds.drop_vars( ["Time_Equation"] )
# Posiciones del analema solar.
ds["Sz"] = ( sin(lat)*sin(ds["Declination"])
    - cos(lat)*cos(ds["Declination"])
    *cos(ds["lon_subs"]-lon) )
ds = ds.drop_vars( ["lon_subs"] )
# Ángulo del cénit solar.
ds["Zenith_Angle"] = acos(ds["Sz"])
ds = ds.drop_vars( "Sz" )
# Ángulo acimutal solar.
ds["Azimuth_Angle"] = acos( ( sin(ds["Declination"])
    - cos(ds["Zenith_Angle"])*sin(lat) )
    / ( sin(ds["Zenith_Angle"])*cos(lat) ) )
ds["Azimuth_Angle"] = ds["Azimuth_Angle"].where(
    ds["Hour_Angle"] < 0, 360 - ds["Azimuth_Angle"] )
ds = ds.drop_vars( ["Declination", "Hour_Angle"] )
# Masa de aire.
ds["Air_Mass"] = ( 1/(cos(ds["Zenith_Angle"])
    + 0.15/(93.885 - ds["Zenith_Angle"])**1.253 )
    * ds["Pressure"]/1013.25 )
ds["Air_Mass"] = ds["Air_Mass"].where( ds["Zenith_Angle"] < 85.5, 0 )
ds = ds.drop_vars( ["Day_Angle", "Pressure"] )

In [5]:
# Casos de orientación de sistemas fotovoltaico

# Casos a estudiar
cases = [ "south_no_track", "west_no_track", "east_no_track",
    "1_track", "2_track", "bifacial_vertical_west_main",
    "bifacial_vertical_east_main", "bifacial_vertical_west_back",
    "bifacial_vertical_east_back", "bifacial_south_back" ]
names_dn = ["South-facing", "West-facing", "East-facing",
    "1 axis tracking", "2 axis tracking", "Bifacial, vertical, west-facing",
    "Bifacial, vertical, east-facing", "Bifacial, south facing", ]
# Variables fotovoltaicas por caso
# Inclinación
track_tilt       =   [ f"{x}_Tilt"               for x in cases ]
# Azimuth
track_azimuth    =   [ f"{x}_Azimuth"            for x in cases ]
# Ángulo entre el panel y el sol, Angle of Incidence
track_AOI        =   [ f"{x}_Angle_of_Incidence" for x in cases ]
# Radiación incidente en el panel [W/m^2], Plane of Array Irradiace
track_POA        =   [ f"{x}_POA"                for x in cases ]
# Producción fotovoltaica por kilowatt de capacidad [W/kWp]
track_P_mp       = ( [ f"{x}_P_mp"               for x in cases ]
    + [ "bifacial_vertical_west_P_mp",
        "bifacial_vertical_east_P_mp",
        "bifacial_south_P_mp" ] )
# Producción para cada caso
prod_n           = track_P_mp[0:5] + track_P_mp[10:]
# Demanda neta [GW]
track_net_demand = ( [ f"{x}_net_demand"         for x in cases[0:5] ]
    + [ "bifacial_vertical_west_net_demand",
        "bifacial_vertical_east_net_demand",
        "bifacial_south_net_demand" ] )
# Factor bifacial
P_bf = [ 1, 1, 1, 1, 1, 1, 1, 0.7, 0.7, 0.7 ]

# Ángulos de orientación de los sistemas
# Orientación del seguidor de un eje
# Asumimos inclinación de 0 grados
azimuth_tracker = 180
# south_no_track
ds[track_azimuth[0]] = 180
ds[track_tilt[0]   ] = lat
# west_no_track
ds[track_azimuth[1]] = 270
ds[track_tilt[1]   ] = lat
# east_no_track
ds[track_azimuth[2]] = 90
ds[track_tilt[2]   ] = lat
# 1_track
ds[track_tilt[3]   ] = np.abs( atan( tan(ds["Zenith_Angle"])
    * sin(ds["Azimuth_Angle"] - azimuth_tracker) ) )
ds[track_azimuth[3]] = 90
ds[track_azimuth[3]] = ds["1_track_Azimuth"
    ].where(ds["Azimuth_Angle"]<180, 270)
# 2_track
ds[track_tilt[4]   ] = ds["Zenith_Angle"]
ds[track_azimuth[4]] = ds["Azimuth_Angle"]
# bifacial_vertical_west_main
ds[track_tilt[5]   ] = 90
ds[track_azimuth[5]] = 270
# bifacial_vertical_east_main
ds[track_tilt[6]   ] = 90
ds[track_azimuth[6]] = 90
# bifacial_vertical_west_back
ds[track_tilt[7]   ] = 90
ds[track_azimuth[7]] = 90
# bifacial_vertical_east_back
ds[track_tilt[8]   ] = 90
ds[track_azimuth[8]] = 270
# bifacial_south_back
ds[track_tilt[9]   ] = 90 + lat
ds[track_azimuth[9]] = 0

In [6]:
# Producción fotovoltaica

# Modelo de Pérez de Cielo Difuso para calcular 
# la radiación en un plano inclinado
# Diffuse Horizontal Radiation.
ds["DHI"] = ds["GHI"] - ds["DNI"] * cos(ds["Zenith_Angle"])
ds["DHI"] = ds["DHI"].where(ds["DHI"]>0, 0.001)
K = 5.535e-6
# Perez clearness bins.
ds["bins"] = 0
ds["bins"] = ds["bins"].where( ds["DHI"] == 0.001,
    ( (ds["DHI"]+ds["DNI"])/ds["DHI"] + K*ds["Zenith_Angle"]**3 )
    / ( 1 + K*ds["Zenith_Angle"]**3 ) )
ds["DHI"] = ds["DHI"].where(ds["DHI"]>0.001, 0)
ds["epsilon"] = ds["bins"   ].where( ds["bins"] < 6.200, 8 )
ds["epsilon"] = ds["epsilon"].where( 
    ~( (ds["bins"]>4.500) & (ds["bins"]<6.200) ), 7 )
ds["epsilon"] = ds["epsilon"].where( 
    ~( (ds["bins"]>2.600) & (ds["bins"]<4.500) ), 6 )
ds["epsilon"] = ds["epsilon"].where( 
    ~( (ds["bins"]>1.950) & (ds["bins"]<2.600) ), 5 )
ds["epsilon"] = ds["epsilon"].where( 
    ~( (ds["bins"]>1.500) & (ds["bins"]<1.950) ), 4 )
ds["epsilon"] = ds["epsilon"].where( 
    ~( (ds["bins"]>1.230) & (ds["bins"]<1.500) ), 3 )
ds["epsilon"] = ds["epsilon"].where( 
    ~( (ds["bins"]>1.065) & (ds["bins"]<1.500) ), 2 )
ds["epsilon"] = ds["epsilon"].where( ds["bins"] > 1.065, 1 )
Perez = pd.read_csv("../code/Perez.csv", index_col = "bin" )
ds = ds.drop_vars( "bins" )
# Extraterrestrial radiation.
Ea = 1367
# Coeficientes
ds["Delta"] = ds["DHI"] * ds["Air_Mass"] / Ea
ds = ds.drop_vars( "Air_Mass" )
for j in Perez.columns:
    ds[j] = 0.0
    for i in Perez.index:
        ds[j] = ds[j].where(ds["epsilon"] != i, Perez.loc[i, j] )
ds = ds.drop_vars( "epsilon" )
ds["F1"] = ( ds["f11"] + ds["f12"]*ds["Delta"]
    + np.radians(ds["Zenith_Angle"])*ds["f13"] )
ds = ds.drop_vars( ["f11", "f12", "f13"] )
ds["F1"] = ds["F1"].where( ds["F1"] < 0, 0 )
ds["F2"] = ( ds["f21"] + ds["f22"]*ds["Delta"]
    + np.radians(ds["Zenith_Angle"])*ds["f23"] )
ds = ds.drop_vars( ["f21", "f22", "f23", "Delta"] )
ds["b"] = cos(ds["Zenith_Angle"])
ds["b"] = ds["b"].where( ds["b"] < cos(85), cos(85) )
# Iteramos para cada caso
for i in range(len(cases)):
    # Ángulo entre el panel y el sol, Angle of Incidence
    ds[track_AOI[i]] = (
        cos(ds["Zenith_Angle"])*cos(ds[track_tilt[i]])
        + sin(ds["Zenith_Angle"])*sin(ds[track_tilt[i]])
        *cos(ds["Azimuth_Angle"]-ds[track_azimuth[i]]) )
    ds[track_AOI[i]] = ds[track_AOI[i]].where( ds[track_AOI[i]] < 1, 1 )
    ds[track_AOI[i]] = ds[track_AOI[i]].where( ds[track_AOI[i]] > -1, -1 )
    ds[track_AOI[i]] = acos(ds[track_AOI[i]])
    ds["a"] = cos(ds[track_AOI[i]])
    ds["a"] = ds["a"].where( ds["a"] < 0, 0 )
    # Radiación difusa.
    ds["I_d"] = ( ds["DHI"] * ( (1-ds["F1"])*((1+cos(ds[track_tilt[i]]))/2)
        + ds["F1"]*ds["a"]/ds["b"] + ds["F2"]*sin(ds[track_tilt[i]]) ) )
    ds["I_d_90"] = ( ds["DHI"] * ((1+cos(ds[track_tilt[i]]))/2) )
    ds["I_d"] = ds["I_d"].where( ds["Zenith_Angle"] < 87.5, ds["I_d_90"] )
    ds = ds.drop_vars( "I_d_90" )
    ds["I_d"] = ds["I_d"].where( ds["Zenith_Angle"] < 90, 0 )
    # Radiación directa.
    ds["I_b"] = ds["DNI"] * cos(ds[track_AOI[i]])
    ds["I_b"] = ds["I_b"].where( ds[track_AOI[i]] < 90, 0 )
    ds = ds.drop_vars( track_AOI[i] )
    # Radiación total en el panel.
    ds[track_POA[i]] = ds["I_b"] + ds["I_d"]
    ds = ds.drop_vars( ["I_b", "I_d"] )

ds = ds.drop_vars( ["F1", "F2", "a", "b", 
    "Zenith_Angle", "Azimuth_Angle", "DHI"] )

# NOCT Cell Temperature Model
T_NOCT    = 44 # °C
# Datos de Panel Canadian Solar 550 W
# Modelo: HiKu6 Mono PERC CS6W-550
I_mp      = 13.2 # A
V_mp      = 41.7 # V
A_m       = 1.134*2.278 # m^2
eff_ref   = I_mp * V_mp / (1000 * A_m)
tau_alpha = 0.9
# Ajuste de viento.
#v = 0.61 # Dos pisos.
v = 0.51 # Un piso.
# Ajuste de montaje.
T_adj = 2   + T_NOCT # Building integrated,
# greater than 3.5 in, or ground/rack mounted
#T_adj = 2  + T_NOCT # 2.5 to 3.5 in
#T_adj = 6  + T_NOCT # 1.5 to 2.5 in
#T_adj = 11 + T_NOCT # 0.5 to 1.5 in
#T_adj = 18 + T_NOCT # less than 0.5 in
# Iteramos para cada caso
for i in range(len(cases)):
    # Temperatura de la celda
    ds[cases[i] + "_Cell_Temperature"] = ( ds["Temperature"]
        + ds[track_POA[i]] / 800 * (T_adj-20)
        * (1-eff_ref/tau_alpha) * ( 9.5 / (5.7+3.8*v*ds["Wind Speed"]) ) )
ds.drop_vars(["Wind Speed", "Temperature"])

# Simple efficiency module model
# Eficiencia por temperatura
eff_T = -0.34
# Pérdidas del sistema
eff_n = [ "Soiling", "Shading", "Snow", "Mismatch",
    "Wiring", "Connections", "Light_Induced_Degradation",
    "Nameplate_Rating", "Age", "Availability" ]
eff = np.array( [0.98, 0.97, 1, 0.98, 0.98,
    0.995, 0.985, 0.99, 1, 0.97] ).prod()
# Eficiencia del inversor
eff_inv = 0.96
# Eficiencia del sistema
eff_sys = eff_ref * eff_inv * eff
# DC to AC Size Ratio
DC_AC = 1.1
# Inverter size
inv_P = I_mp * V_mp / DC_AC

# Iteramos para cada caso
for i in range(len(cases)):
    # Potencia generada en AC
    ds[track_P_mp[i]] = ( P_bf[i] * ds[track_POA[i]]*eff_sys*A_m *
        ( 1 + eff_T/100 * (ds[cases[i] + "_Cell_Temperature"]-25) ) )
    ds[track_P_mp[i]] = ds[ track_P_mp[i] ].where(
        ds[track_P_mp[i]] < inv_P, inv_P )
    # El resultando es la generación por cada kWp.
    ds[track_P_mp[i]] = ds[track_P_mp[i]] * 1000 / ( I_mp * V_mp )
    ds[track_P_mp[i]] = ds[track_P_mp[i]].where(
        ds[track_POA[i]] > 0, 0 ).where(ds["GHI"] > 0, 0)
    ds = ds.drop_vars( cases[i] + "_Cell_Temperature" )
    ds = ds.drop_vars( track_POA[i] )

# Calculamos la producción bifacial total
ds[track_P_mp[10]] = ds[track_P_mp[5]] + ds[track_P_mp[7]]
ds[track_P_mp[11]] = ds[track_P_mp[6]] + ds[track_P_mp[8]]
ds[track_P_mp[12]] = ds[track_P_mp[0]] + ds[track_P_mp[9]]
ds = ds.drop_vars(track_P_mp[5:10])

#print("Photovoltaic production")
#pd.options.display.float_format = "{:,.0f} kWh/kWp".format
#a = ( ds[prod_n] / 2 ).sum(axis = 0)
#a.index = names_dn
#print(a.sort_values(ascending = False).to_string())

In [16]:
# Cargamos la demanda

# Ruta de la demanda para México
path_f = "../data/Demanda/Mexico/Original/"

# Cargamos cada archivo
df_i = []
for f in os.listdir(path_f):
    if f[0] != ".":
        df_0 = pd.read_csv(path_f + f, skiprows = 8,
            na_values = "               ---" )
        # Asignamos columnas
        df_0.columns = ['Sistema', 'Area', 'Hora', 'Generacion (MWh)',
           'Importacion Total (MWh)', 'Exportacion Total (MWh)',
            'Intercambio neto entre Gerencias (MWh)',
            'Estimacion de Demanda por Balance (MWh)']
        df_0["Area"] = df_0["Area"].where(df_0["Area"]!="BCA", df_0["Sistema"]) 
        df_0 = df_0.drop(columns = "Sistema")
        # Damos formato a la fecha
        df_0["Hora"] = df_0["Hora"].where(df_0["Hora"] < 25, np.nan)
        #df_0 = df_0.dropna()
        df_0["Hora"] = df_0["Hora"].where(df_0["Hora"] < 24, 0)
        df_0["Date"] = pd.to_datetime( df_0["Hora"].apply(
            lambda x: f"{f[40:50]} {x:02.0f}:00:00" ) )
        
        df_i.append(df_0.rename(columns = {"Date": "time"}).set_index("time"))

# Unimos todos los archivos
demanda = "Estimacion de Demanda por Balance (MWh)"
a = pd.concat(df_i)
a["NUM_CENACE"] = a["Area"].apply(NUM_CENACE)
a = a.set_index( "NUM_CENACE", append = True)[[demanda]].sort_index()
ds_d = a.loc[ ~a.index.duplicated() ].to_xarray()
ds_d = ds_d.rename_vars({demanda: "Demand"})
#ds_d = ds_d.interp_like(ds)

ds_d

<xarray.Dataset> Size: 701kB
Dimensions:     (time: 8760, NUM_CENACE: 9)
Coordinates:
  * time        (time) datetime64[ns] 70kB 2022-01-01 ... 2022-12-31T23:00:00
  * NUM_CENACE  (NUM_CENACE) int64 72B 0 1 2 3 4 5 6 7 8
Data variables:
    Demand      (time, NUM_CENACE) float64 631kB 1.149e+03 219.2 ... 1.457e+03

In [141]:
# Minimun capacity to reach zero net demand during duck's belly
ds_n = ds_d.isel({"time": 0, "NUM_CENACE": ds["NUM_CENACE"]}).copy()
for v in prod_n: ds_n[v] = ds_n["Demand"].copy()
ds_n = ds_n.drop_vars(["Demand", "time"])
# Función que determina la demanda mínima en términos de la capacidad
def demand(x, v, c):
    return np.nanmin((ds_d["Demand"] - ds[v] * x).sel(NUM_CENACE = c))
# Encontramos la capacidad que ocasiona una demanda nula
for v in prod_n:
    n = []
    for c in ds["NUM_CENACE"].values:
        n.append( optimize.fsolve(demand, x0 = 5, args = (v, c))[0] )
    ds_n[v] = ("NUM_CENACE", n)

ds_n

<xarray.Dataset> Size: 144B
Dimensions:                      (NUM_CENACE: 2)
Coordinates:
  * NUM_CENACE                   (NUM_CENACE) int64 16B 2 7
Data variables:
    south_no_track_P_mp          (NUM_CENACE) float64 16B 6.321 6.035
    west_no_track_P_mp           (NUM_CENACE) float64 16B 7.985 7.361
    east_no_track_P_mp           (NUM_CENACE) float64 16B 6.795 6.652
    1_track_P_mp                 (NUM_CENACE) float64 16B 6.336 6.349
    2_track_P_mp                 (NUM_CENACE) float64 16B 5.392 5.406
    bifacial_vertical_west_P_mp  (NUM_CENACE) float64 16B 8.608 8.746
    bifacial_vertical_east_P_mp  (NUM_CENACE) float64 16B 6.585 6.768
    bifacial_south_P_mp          (NUM_CENACE) float64 16B 5.967 5.654

In [ ]:
# Minimun capacity to reach zero net demand during duck's belly
# Función que determina la demanda mínima en términos de la capacidad
def demand(x, v): return np.nanmin(df[["Demand"]].values - df[[v]].values * x)
# Encontramos la capacidad que ocasiona una demanda nula
n = []
for v in prod_n: n.append( optimize.fsolve(demand, x0 = 5, args = (v))[0] )
# Resultados
print("Minimun capacity to reach zero net demand during duck's belly")
pd.options.display.float_format = "{:,.2f} GWp".format
cap = pd.Series( n, index = names_dn )
print( cap.sort_values( ascending = True ).to_string() )

# Energy produced using capacity that produces zero demand
df = df.copy()
# Net demand with capacity that produces zero demand
df[track_net_demand] = ( df[["Demand"]].values
    - df[prod_n].values * n ).copy()
prod_dn_min = [f"{x}_min" for x in track_net_demand]
# Net demand with same capacity for all cases
df[prod_dn_min] = ( df[["Demand"]].values
    - df[prod_n].values * np.min(n) )
print("\nEnergy produced using capacity that produces zero demand")
pd.options.display.float_format = "{:,.0f} MWh".format
ener = pd.Series( ( df[prod_n] * n / 2
    ).copy().sum().values, index = names_dn )
print( ener.sort_values(ascending = False).to_string() )

# Demand ramp
track_net_demand_dt = [ f"{x}_dt" for x in track_net_demand ]
df[track_net_demand_dt] = ( ( df[track_net_demand]
    - df[track_net_demand].shift(1) ) * 2)
df["Demand_dt"] = ( ( df["Demand"]
    - df["Demand"].shift(1) ) * 2).copy()

# Capacity factor
print( "\nCapacity factor" )
pd.options.display.float_format = "{:,.1f} %".format
cap_f = pd.Series( 100 * ener.values / ( 1000 * 24 * 365 * cap.values ),
    index = names_dn )
print( cap_f.sort_values(ascending = False).to_string() )